# Neural Probabilistic Language Model

In this notebook, we'll implement the [Neural Probabilistic Language Model (Bengio et al. 2003)](http://machinelearning.wustl.edu/mlpapers/paper_files/BengioDVJ03.pdf). This model was one of the first applications of deep learning to NLP, predating `word2vec` by a whole decade! Many of the ideas in this model, however, are remarkably current - such as the skip-layer connection now popular in the form of "residual networks."

This model is a straightforward extension of n-gram language modeling: it uses a fixed context window, but instead of a table it uses a neural network to predict the next word. It'll also serve as a segue to Assignment 4, in which you'll implement a recurrent neural network language model (RNNLM).

#### Note on training time
The NPLM can take a while to train on a slower machine - we clocked it at 10-20 min on a 2-core Cloud Compute instance. 

If you're using a cloud compute instance, you can add more CPUs without having to re-do setup. With your instance turned off, go to https://console.cloud.google.com/compute/instances, click your instance, and go to "Edit". Under machine type, select "Custom" and pick 4-8 CPUs and 2 GB of RAM. Make sure you shut down when you're done, and use the Edit menu again to scale back the size to something less expensive.

In [1]:
import os, sys, re, json, time, shutil
import itertools, collections
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz

## NPLM Model Architecture

Recall that our n-gram mode of order $k+1$ was:

$$ P(w_i | w_{i-1}, w_{i-2}, ..., w_0) \approx P(w_i | w_{i-1}, ..., w_{i-k}) $$

Where we estimated the probabilities by smoothed maximum likelihood.

For the NPLM, we'll replace that estimate with a neural network predictor that directly learns a mapping from contexts $(w_{i-1}, ..., w_{i-k})$ to a distribution over words $w_i$:

$$ P(w_i | w_{i-1}, ..., w_{i-k}) = f(w_i, (w_{i-1}, ..., w_{i-k})) $$

Here's what that network will look like:
![NPLM architecture](nplm.png)

Broadly, there are three parts:
1. **Embedding layer**: map words into vector space
2. **Hidden layer**: compress and apply nonlinearity
3. **Output layer**: predict next word using softmax

The model also has *skip connections* between the embedding layer and the output layer. This just means that the output layer takes as input the concatenated embeddings in addition to the hidden layer output. This was considered an unusual pattern, but has recently become popular again in the form of [Residual Networks](http://www.kaiminghe.com/icml16tutorial/icml2016_tutorial_deep_residual_networks_kaiminghe.pdf) and [Highway Networks](https://arxiv.org/abs/1505.00387).

With modern computers and a couple tricks, we should be able to get a decent model to run in just a few minutes - a far cry from the three weeks it took in 2003!

# Constructing our Model

To implement the NPLM in TensorFlow, we need to define a Tensor for each model component. To make a clear distinction between TF and non-TF code, we'll use variable names that end in an underscore for Tensor objects. We'll also construct the model so it can accept batch inputs, as this will greatly speed up training.

Hyperparameters:
- `V` : vocabulary size
- `M` : embedding size
- `N` : context window size
- `H` : hidden units

Inputs:
- `ids_` : (batch_size, N), integer indices for context words
- `y_` : (batch_size,), integer indices for target word

Model parameters:
- `C_` : (V,M), input-side word embeddings
- `W1_` : (NxM, H)
- `b1_` : (H,)
- `W2_` : (H, V)
- `W3_` : (NxM, V), matrix for skip-layer connection
- `b3_` : (V,)

Intermediate states:
- `x_` : (batch_size, NxM), concatenated embeddings
- `h_` : (batch_size, H), hidden state $= \tanh(xW_1 + b_1)$
- `logit_` : (batch_size, V), $= hW_2 + xW_3 + b_3$

In [2]:
tf.reset_default_graph()
tf.set_random_seed(42)

##
# Hyperparameters
V = 10000
M = 30
N = 3
H = 50

# Inputs
# Using "None" in place of batch size allows 
# it to be dynamically computed later.
with tf.name_scope("Inputs"):
    ids_ = tf.placeholder(tf.int32, shape=[None, N], name="ids")
    y_ = tf.placeholder(tf.int32, shape=[None], name="y")
    
with tf.name_scope("Embedding_Layer"):
    C_ = tf.Variable(tf.random_uniform([V, M], -1.0, 1.0), name="C")
    # embedding_lookup gives shape (batch_size, N, M)
    x_ = tf.reshape(tf.nn.embedding_lookup(C_, ids_), 
                    [-1, N*M], name="x")
    
with tf.name_scope("Hidden_Layer"):
    W1_ = tf.Variable(tf.random_normal([N*M,H]), name="W1")
    b1_ = tf.Variable(tf.zeros([H,], dtype=tf.float32), name="b1")
    h_ = tf.tanh(tf.matmul(x_, W1_) + b1_, name="h")
    
with tf.name_scope("Output_Layer"):
    W2_ = tf.Variable(tf.random_normal([H,V]), name="W2")
    W3_ = tf.Variable(tf.random_normal([N*M,V]), name="W3")
    b3_ = tf.Variable(tf.zeros([V,], dtype=tf.float32), name="b3")
    # Concat [h x] and [W2 W3]
    hx_ = tf.concat([h_, x_], 1, name="hx")
    W23_ = tf.concat([W2_, W3_], 0, name="W23")
    logits_ = tf.add(tf.matmul(hx_, W23_), b3_, name="logits")

We'll add in our usual cross-entropy loss. Recall from async that this is *very* slow for a large vocabulary, and even for a small vocabulary it represents the bulk of the computation time. To speed up training we'll use a sampled softmax loss, as in [Jozefowicz et al. 2016](https://arxiv.org/abs/1602.02410):

In [3]:
with tf.name_scope("Cost_Function"):
    # Sampled softmax loss, for training
    per_example_train_loss_ = tf.nn.sampled_softmax_loss(weights=tf.transpose(W23_), biases=b3_,
                                             labels=tf.expand_dims(y_, 1), inputs=hx_,
                                             num_sampled=100, num_classes=V,
                                             name="per_example_sampled_softmax_loss")
    train_loss_ = tf.reduce_mean(per_example_train_loss_, name="sampled_softmax_loss")
    
    # Full softmax loss, for scoring
    per_example_loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=logits_, name="per_example_loss")
    loss_ = tf.reduce_mean(per_example_loss_, name="loss")

And add training ops. We'll use AdaGrad instead of vanilla SGD, as this tends to converge faster:

In [4]:
with tf.name_scope("Training"):
    alpha_ = tf.placeholder(tf.float32, name="learning_rate")
    optimizer_ = tf.train.AdagradOptimizer(alpha_)
    # train_step_ = optimizer_.minimize(loss_)
    train_step_ = optimizer_.minimize(train_loss_)
    
# Initializer step
init_ = tf.global_variables_initializer()

Finally, we'll add a few ops to do prediction:
- `pred_proba_` : (batch_size, V), $ = P(w_i | w_{i-1}, ...)$ for all words $i$
- `pred_max` : (batch_size,), id of most likely next word
- `pred_random` : (batch_size,), id of a randomly-sampled next word

In [5]:
with tf.name_scope("Prediction"):
    pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
    pred_max_ = tf.argmax(logits_, 1, name="pred_max")
    pred_random_ = tf.multinomial(logits_, 1, name="pred_random")

We can use TensorBoard to view this graph, even before we run the model:

In [6]:
summary_writer = tf.summary.FileWriter("tf_graph", 
                                       tf.get_default_graph())

In a separate terminal, run:
```
tensorboard --logdir="~/w266/materials/week4/tf_graph" --port 6006
```
and go to http://localhost:6006/

It should look something like this:
![NPLM graph](nplm-graph.png)

## Loading the Corpus

As in the original paper, we'll train on the Brown corpus. We'll pre-process the inputs as on Assignment 2, lowercasing and canonicalizing digits and adding `<s>` and `</s>` markers to sentence boundaries. See [embeddings.ipynb](../embeddings/embeddings.ipynb) for more details.

We'll also restrict our vocabulary to the top 10,000 words. 

_**Exercise:**_ why do we need a fixed-size vocabulary for our neural model? And why does it help to restrict its size?

In [7]:
corpus_name = "brown"
V = 10000

vocab, train_ids, test_ids = utils.load_corpus(corpus_name, split=0.8, V=V, shuffle=False)

[nltk_data] Downloading package brown to /home/iftenney/nltk_data...
[nltk_data]   Package brown is already up-to-date!
Vocabulary: 10,000 words
Loaded 57,340 sentences (1.16119e+06 tokens)
Training set: 45,872 sentences (979,646 tokens)
Test set: 11,468 sentences (181,546 tokens)


Our model is designed to accept batches of data, so we need to do a little re-formatting. We want our input batches to look like the following, where the first $N$ columns are the inputs and the last is the target word:

In [8]:
cols = ["w_{i-%d}" % d for d in range(N,0,-1)] + ["target: w_i"]
M = np.array([[0,3,5613,655], [3,5613,655,2288], [5613,655,2288,1640]])
utils.pretty_print_matrix(M, cols=cols, dtype=int)

,w_{i-3},w_{i-2},w_{i-1},target: w_i
0,0,3,5613,655
1,3,5613,655,2288
2,5613,655,2288,1640


We'll format our entire corpus like this, and then we can just sample blocks from it to get our training minibatches. The code for this is in `utils.py`, but you can view it in-notebook by running the cell below:

In [9]:
utils.build_windows??

In [12]:
train_windows = utils.build_windows(train_ids, N)
test_windows = utils.build_windows(test_ids, N)

# Check that we got what we want
# Just look at the first few IDs for this sample
utils.pretty_print_matrix(utils.build_windows(train_ids[:(N+5)], N, shuffle=False), cols=cols, dtype=int)

,w_{i-3},w_{i-2},w_{i-1},target: w_i
0,0,3,5613,655
1,3,5613,655,2288
2,5613,655,2288,1640
3,655,2288,1640,65
4,2288,1640,65,1843


## Training time!

With our data in array form, we can train our model much like any machine learning model. The code below should look familiar to the `train_nn` function from Assignment 1. We'll factor out a few operations into helpers, so that the basic structure is clearer.

In [13]:
##
# Helper functions for training
def train_batch(session, batch, alpha):
    # Feed last column as targets
    feed_dict = {ids_:batch[:,:-1],
                 y_:batch[:,-1],
                 alpha_:alpha}
    c, _ = session.run([train_loss_, train_step_],
                       feed_dict=feed_dict)
    return c

## We'll use this to generate the batches.
utils.batch_generator??

Training a single epoch should take around 6-7 minutes on a 2-core Cloud Compute instance, or around 30 seconds on a GTX 980 GPU. You should get good results after just 2-3 epochs (train loss around 3.5).

Rememer that the cost printed is the average *training* loss. Since we're using the sampled softmax, this will be an underestimate of the true loss. We'll need to do a separate run over the data to compute perplexity.

In [14]:
# One epoch = one pass through the training data
num_epochs = 3
batch_size = 100
alpha = 0.5  # learning rate
print_every = 1000

np.random.seed(42)

session = tf.Session()
session.run(init_)

t0 = time.time()
for epoch in xrange(1,num_epochs+1):
    t0_epoch = time.time()
    epoch_cost = 0.0
    total_batches = 0
    print ""
    for i, batch in enumerate(utils.batch_generator(train_windows, batch_size)):
        if (i % print_every == 0):
            print "[epoch %d] seen %d minibatches" % (epoch, i)
        
        epoch_cost += train_batch(session, batch, alpha)
        total_batches = i + 1

    avg_cost = epoch_cost / total_batches
    print "[epoch %d] Completed %d minibatches in %s" % (epoch, i, utils.pretty_timedelta(since=t0_epoch))
    print "[epoch %d] Average cost: %.03f" % (epoch, avg_cost,)


[epoch 1] seen 0 minibatches
[epoch 1] seen 1000 minibatches
[epoch 1] seen 2000 minibatches
[epoch 1] seen 3000 minibatches
[epoch 1] seen 4000 minibatches
[epoch 1] seen 5000 minibatches
[epoch 1] seen 6000 minibatches
[epoch 1] seen 7000 minibatches
[epoch 1] seen 8000 minibatches
[epoch 1] seen 9000 minibatches
[epoch 1] seen 10000 minibatches
[epoch 1] Completed 10255 minibatches in 0:04:25
[epoch 1] Average cost: 5.745

[epoch 2] seen 0 minibatches
[epoch 2] seen 1000 minibatches
[epoch 2] seen 2000 minibatches
[epoch 2] seen 3000 minibatches
[epoch 2] seen 4000 minibatches
[epoch 2] seen 5000 minibatches
[epoch 2] seen 6000 minibatches
[epoch 2] seen 7000 minibatches
[epoch 2] seen 8000 minibatches
[epoch 2] seen 9000 minibatches
[epoch 2] seen 10000 minibatches
[epoch 2] Completed 10255 minibatches in 0:04:30
[epoch 2] Average cost: 4.057

[epoch 3] seen 0 minibatches
[epoch 3] seen 1000 minibatches
[epoch 3] seen 2000 minibatches
[epoch 3] seen 3000 minibatches
[epoch 3] seen

# Scoring

We'll score our model the same as the n-gram model, by computing perplexity over the dev set. Recall that perplexity is just the exponentiated average cross-entropy loss:

$$ \text{Perplexity} = \left( \prod_i \frac{1}{Q(x_i)} \right)^{1/N} = \left( \prod_i 2^{- \log_2 Q(x_i)} \right)^{1/N} = 2^{\left(\frac{1}{N} \sum_i -\log_2 Q(x_i)\right)} = 2^{\tilde{CE}(P,Q)}$$

In practice TF uses the natural log, so the loss will be scaled by a factor of $\ln 2$ - but the base cancels and the perplexity scores will be the same.

Note that below we use `loss_`, which is the cross-entropy loss with the full softmax. Because this is so much slower than the sampled softmax, on a slower machine the scoring step might take as long or longer than training!

In [15]:
def score_batch(session, batch):
    feed_dict = {ids_:batch[:,:-1],
                 y_:batch[:,-1]}
    return session.run(loss_, feed_dict=feed_dict)

def score_dataset(data):
    total_cost = 0.0
    total_batches = 0
    for batch in utils.batch_generator(data, 1000):
        total_cost += score_batch(session, batch)
        total_batches += 1

    return total_cost / total_batches

In [16]:
print "Train set perplexity: %.03f" % np.exp(score_dataset(train_windows))
print "Test set perplexity: %.03f" % np.exp(score_dataset(test_windows))

Train set perplexity: 256.324
Test set perplexity: 292.498


Looks pretty good! Note that these numbers aren't directly comparable to the literature, since we made the task easier by lowercasing everything, canonicalizing digits, and treating a fairly large number of words as an `<unk>` token.

We can remove some of this handicap by looking at our perplexity on non-`<unk>` target words:

In [17]:
filtered_test_windows = test_windows[test_windows[:,-1] != vocab.UNK_ID]
print "Filtered test set perplexity: %.03f" % np.exp(score_dataset(filtered_test_windows))

Filtered test set perplexity: 358.668


# Sampling

We can sample sentences from the model much as we did with n-gram models. We'll use the `pred_random_` op that we defined before:

In [18]:
def predict_next(session, seq):
    feed_dict={ids_:np.array([seq[-N:]])}
    next_id = session.run(pred_random_, feed_dict=feed_dict)
    return next_id[0][0]

def score_seq(session, seq):
    # Some gymnastics to generate windows for scoring
    windows = [seq[i:i+N+1] for i in range(len(seq)-(N+1))]
    return score_batch(session, np.array(windows))

max_length = 30
num_sentences = 5

for _ in range(num_sentences):
    seq = [vocab.word_to_id["<s>"]]*N  # init N+1-gram model
    for i in range(max_length):
        seq.append(predict_next(session, seq))
        if seq[-1] == vocab.word_to_id["<s>"]: break
    print " ".join(vocab.ids_to_words(seq))
    score = score_seq(session, seq)
    print "[%d tokens; log P(seq): %.02f, per-token: %.02f]" % (len(seq), score, 
                                                                score/(len(seq)-2))
    print ""

<s> <s> <s> <unk> has become a unless not not rifle to <unk> up in spirit . <s>
[18 tokens; log P(seq): 4.37, per-token: 0.27]

<s> <s> <s> myself . `` i never <unk> . <s>
[11 tokens; log P(seq): 4.35, per-token: 0.48]

<s> <s> <s> put from each meeting this <unk> and <unk> with the more motors used . <s>
[18 tokens; log P(seq): 4.82, per-token: 0.30]

<s> <s> <s> the entire regulations planted dog from that a ground and garibaldi's hard leaves a draft modern public never practice at the born of the plane burst from almost religious home
[33 tokens; log P(seq): 6.02, per-token: 0.19]

<s> <s> <s> <unk> DG over DGDGDGDG under the ground of <unk> on changed at areas . <s>
[18 tokens; log P(seq): 4.52, per-token: 0.28]

